In [10]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'Zookeeper'
output_file = f'outputs/parser/LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full/{dataset}_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
# print(a)
df1 = pd.read_csv(output_file)
logs1 = df1['EventTemplate'].tolist()
df2 = pd.read_csv(groundtruth_file)
logs2 = df2['EventTemplate'].tolist()

count = 0
length = len(logs1)
for i in tqdm(range(length)):
    if logs1[i] == logs2[i]:
        count += 1
print(count / length)
print(count)


100%|██████████| 74273/74273 [00:00<00:00, 3533655.57it/s]

0.9626647637768772
71500


In [ ]:
from utils.postprocess import correct_single_template

print(correct_single_template('<*> "<*> <*> 1.2.3.4:<*> status: <*> len: <*> time: <*>'))

In [3]:
import re
import pandas as pd
dataset = 'HDFS'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
df = pd.read_csv(groundtruth_file)
# 定义更新规则的函数
def update_content(content):
    # 使用正则表达式匹配并更新内容
    pattern = r'(BLOCK\* NameSystem\.allocateBlock: [^ ]*) ([^ ]*)'
    replacement = r'\1. \2'
    return re.sub(pattern, replacement, content)

# 更新Content列
df['Content'] = df['Content'].apply(update_content)

# 保存修改后的CSV文件
df.to_csv('updated_file.csv', index=False)